In [1]:
#carrier_delay

In [2]:
#weather_delay
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [3]:
# Load the dataset
df = pd.read_csv('Airline_Delay_Cause.csv')

In [4]:
# Clean the dataset by dropping irrelevant columns
delay_df_clean = df.drop(['year', 'carrier', 'airport', 'arr_del15','weather_delay','arr_delay','nas_delay','security_delay','late_aircraft_delay'], axis=1)
delay_df_clean.head()

,month,carrier_name,airport_name,arr_flights,carrier_ct,weather_ct,nas_ct,security_ct,late_aircraft_ct,arr_cancelled,arr_diverted,carrier_delay
0,9,Endeavor Air Inc.,"Allentown/Bethlehem/Easton, PA: Lehigh Valley ...",81.0,5.61,0.56,0.84,0.0,0.00,0.0,0.0,201.0
1,9,Endeavor Air Inc.,"Alexandria, LA: Alexandria International",81.0,4.43,0.00,1.13,0.0,0.44,0.0,0.0,440.0
2,9,Endeavor Air Inc.,"Augusta, GA: Augusta Regional at Bush Field",133.0,3.49,2.04,3.84,0.0,2.63,10.0,0.0,1269.0
3,9,Endeavor Air Inc.,"Albany, NY: Albany International",73.0,0.82,1.00,0.23,0.0,1.95,0.0,0.0,28.0
4,9,Endeavor Air Inc.,"Atlanta, GA: Hartsfield-Jackson Atlanta Intern...",2204.0,61.70,17.36,88.36,0.0,112.57,40.0,4.0,7840.0


In [5]:
row_count = delay_df_clean.count()

print(row_count)

clean_delay_df_new = delay_df_clean.dropna()

new_row_count = clean_delay_df_new.count()

print(new_row_count)

month               370758
carrier_name        370758
airport_name        370758
arr_flights         370121
carrier_ct          370121
weather_ct          370121
nas_ct              370121
security_ct         370121
late_aircraft_ct    370121
arr_cancelled       370121
arr_diverted        370121
carrier_delay       370121
dtype: int64
month               370121
carrier_name        370121
airport_name        370121
arr_flights         370121
carrier_ct          370121
weather_ct          370121
nas_ct              370121
security_ct         370121
late_aircraft_ct    370121
arr_cancelled       370121
arr_diverted        370121
carrier_delay       370121
dtype: int64


In [6]:
# Identify the top 20 airports by count
top_20_airports = clean_delay_df_new['airport_name'].value_counts().head(20).index

# Replace less frequent airports with 'Other'
clean_delay_df_new['airport_name'] = clean_delay_df_new['airport_name'].apply(
    lambda x: x if x in top_20_airports else 'Other'
)

# Verify the changes
print(clean_delay_df_new['airport_name'].value_counts())

airport_name
Other                                                         305790
Detroit, MI: Detroit Metro Wayne County                         3437
Pittsburgh, PA: Pittsburgh International                        3367
Indianapolis, IN: Indianapolis International                    3362
Kansas City, MO: Kansas City International                      3349
Raleigh/Durham, NC: Raleigh-Durham International                3252
Austin, TX: Austin - Bergstrom International                    3231
Washington, DC: Ronald Reagan Washington National               3222
Chicago, IL: Chicago O'Hare International                       3216
Atlanta, GA: Hartsfield-Jackson Atlanta International           3210
St. Louis, MO: St Louis Lambert International                   3206
Philadelphia, PA: Philadelphia International                    3191
Minneapolis, MN: Minneapolis-St Paul International              3184
Phoenix, AZ: Phoenix Sky Harbor International                   3176
Boston, MA: Logan Int

C:\Users\18056\AppData\Local\Temp\ipykernel_19800\83557759.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_delay_df_new['airport_name'] = clean_delay_df_new['airport_name'].apply(


In [7]:
# Calculate the IQR for the target variable (arr_delay)
Q1 = clean_delay_df_new['carrier_delay'].quantile(0.25)
Q3 = clean_delay_df_new['carrier_delay'].quantile(0.75)
IQR = Q3 - Q1

# Define lower and upper bounds
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter out the outliers from the entire dataset based on arr_delay
clean_delay_df_new = clean_delay_df_new[(clean_delay_df_new['carrier_delay'] >= lower_bound) & 
                                               (clean_delay_df_new['carrier_delay'] <= upper_bound)]

In [8]:
# Verify the new row count and the first few rows
print(clean_delay_df_new.count())
print(clean_delay_df_new.head())

month               331093
carrier_name        331093
airport_name        331093
arr_flights         331093
carrier_ct          331093
weather_ct          331093
nas_ct              331093
security_ct         331093
late_aircraft_ct    331093
arr_cancelled       331093
arr_diverted        331093
carrier_delay       331093
dtype: int64
   month       carrier_name airport_name  arr_flights  carrier_ct  weather_ct  \
0      9  Endeavor Air Inc.        Other         81.0        5.61        0.56   
1      9  Endeavor Air Inc.        Other         81.0        4.43        0.00   
2      9  Endeavor Air Inc.        Other        133.0        3.49        2.04   
3      9  Endeavor Air Inc.        Other         73.0        0.82        1.00   
5      9  Endeavor Air Inc.        Other          2.0        0.98        0.00   

   nas_ct  security_ct  late_aircraft_ct  arr_cancelled  arr_diverted  \
0    0.84          0.0              0.00            0.0           0.0   
1    1.13          0.0        

In [9]:
clean_delay_df_new.to_csv('carrier_delay.csv', index=False)

In [10]:
# features
X = clean_delay_df_new.drop(columns='carrier_delay') 

#target output
y = clean_delay_df_new['carrier_delay'] 

print(X)
print(y)

        month            carrier_name  \
0           9       Endeavor Air Inc.   
1           9       Endeavor Air Inc.   
2           9       Endeavor Air Inc.   
3           9       Endeavor Air Inc.   
5           9       Endeavor Air Inc.   
...       ...                     ...   
370753      9  Southwest Airlines Co.   
370754      9  Southwest Airlines Co.   
370755      9  Southwest Airlines Co.   
370756      9  Southwest Airlines Co.   
370757      9  Southwest Airlines Co.   

                                         airport_name  arr_flights  \
0                                               Other         81.0   
1                                               Other         81.0   
2                                               Other        133.0   
3                                               Other         73.0   
5                                               Other          2.0   
...                                               ...          ...   
370753           

In [11]:
print(X.select_dtypes(include='object').nunique())

carrier_name    51
airport_name    21
dtype: int64


In [12]:
# Apply one-hot encoding to categorical columns
categorical_columns = ['airport_name', 'carrier_name']  # Add other categorical columns if needed
X_encoded = pd.get_dummies(X, columns=categorical_columns, drop_first=True)

# Verify the shape of the encoded dataset
print(X_encoded.shape)

(331093, 79)


In [13]:
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=1)

In [14]:
model = RandomForestRegressor(random_state=1)
model.fit(X_train, y_train)

RandomForestRegressor(random_state=1)

In [15]:
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

print(f"Mean Squared Error: {mse}")

Mean Squared Error: 42947.89162971106


In [16]:
carrier_names = clean_delay_df_new['carrier_name'].unique()

print(carrier_names)

['Endeavor Air Inc.' 'American Airlines Network' 'Alaska Airlines Network'
 'JetBlue Airways' 'CommuteAir LLC dba CommuteAir'
 'Delta Air Lines Network' 'Frontier Airlines' 'Allegiant Air'
 'GoJet Airlines LLC d/b/a United Express' 'Hawaiian Airlines Network'
 'Envoy Air' 'Spirit Airlines' 'PSA Airlines Inc.' 'SkyWest Airlines Inc.'
 'Piedmont Airlines' 'Horizon Air' 'United Air Lines Network'
 'Southwest Airlines' 'Mesa Airlines Inc.' 'Republic Airline'
 'Air Wisconsin Airlines Corp' 'Commutair Aka Champlain Enterprises, Inc.'
 'Empire Airlines Inc.' 'ExpressJet Airlines LLC' 'Trans States Airlines'
 'Compass Airlines' 'Peninsula Airways Inc.' 'ExpressJet Airlines Inc.'
 'Cape Air' 'Virgin America' 'American Airlines Inc.'
 'Alaska Airlines Inc.' 'Delta Air Lines Inc.' 'United Air Lines Inc.'
 'Southwest Airlines Co.' 'Frontier Airlines Inc.'
 'Hawaiian Airlines Inc.' 'Spirit Air Lines' 'US Airways Inc.'
 'AirTran Airways Corporation' 'American Eagle Airlines Inc.'
 'Pinnacle Airlines

In [17]:
airport_names = clean_delay_df_new['airport_name'].unique()

print(airport_names)

['Other' 'Nashville, TN: Nashville International'
 'Cleveland, OH: Cleveland-Hopkins International'
 'Washington, DC: Ronald Reagan Washington National'
 'Detroit, MI: Detroit Metro Wayne County'
 'Indianapolis, IN: Indianapolis International'
 'Kansas City, MO: Kansas City International'
 'Minneapolis, MN: Minneapolis-St Paul International'
 'Pittsburgh, PA: Pittsburgh International'
 'Raleigh/Durham, NC: Raleigh-Durham International'
 'St. Louis, MO: St Louis Lambert International'
 'New Orleans, LA: Louis Armstrong New Orleans International'
 'Atlanta, GA: Hartsfield-Jackson Atlanta International'
 'Austin, TX: Austin - Bergstrom International'
 'Boston, MA: Logan International' 'Denver, CO: Denver International'
 'Los Angeles, CA: Los Angeles International'
 "Chicago, IL: Chicago O'Hare International"
 'Philadelphia, PA: Philadelphia International'
 'Phoenix, AZ: Phoenix Sky Harbor International'
 'San Diego, CA: San Diego International']


In [18]:
def predict_delay(month_input, carrier_name_input, airport_name_input):
    # Filter the DataFrame for the selected month
    monthly_data = clean_delay_df_new[clean_delay_df_new['month'] == month_input]
    
    if monthly_data.empty:
        raise ValueError("No data available for the given month.")
    
    # Calculate the mean for numeric columns only
    numeric_columns = monthly_data.select_dtypes(include=['number'])
    mean_values = numeric_columns.mean()
    
    # Prepare the input row with numeric means
    input_data = {
        'month': month_input,
        'carrier_name': carrier_name_input,
        'airport_name': airport_name_input
    }
    # Add mean values for numeric columns
    input_data.update(mean_values.to_dict())

    # Create a prediction (example: predict based on mean delay)
    # Replace this logic with your model or prediction method
    predicted_delay = mean_values['carrier_delay'] if 'carrier_delay' in mean_values else 0
    return predicted_delay

# User input
try:
    month = int(input("Enter the month (as a number, e.g., 1 for January): "))
    carrier_name = input("Enter the carrier name: ")
    airport_name = input("Enter the airport name: ")

    predicted_delay = predict_delay(month, carrier_name, airport_name)
    print(f"Predicted Arrival Delay: {predicted_delay:.2f} minutes")
except ValueError as e:
    print(f"Error: {e}")

Enter the month (as a number, e.g., 1 for January):  1
Enter the carrier name:  Endeavor Air Inc.
Enter the airport name:  San Diego, CA: San Diego International


Predicted Arrival Delay: 540.28 minutes


In [20]:
check_target_mean = clean_delay_df_new['carrier_delay'].mean()

print(check_target_mean)

536.8606735871795


In [21]:
r2_score = model.score(X_test, y_test)
print(f"R^2 Score: {r2_score:.2f}")

R^2 Score: 0.87


In [22]:
import pandas as pd
import itertools

# Get unique values for month, carrier, and airport
months = clean_delay_df_new['month'].unique()
airports = clean_delay_df_new['airport_name'].unique()
carriers = clean_delay_df_new['carrier_name'].unique()

# Generate all combinations of month, airport, and carrier
combinations = list(itertools.product(months, carriers, airports))

# Create a DataFrame to hold the results
predictions = []

for month, carrier, airport in combinations:
    # Create an input row for prediction (mean of other columns)
    input_data = {
        'month': month,
        'carrier_name': carrier,
        'airport_name': airport
    }
    
    # Use the model to predict the delay (could use more advanced logic here)
    predicted_delay = predict_delay(month, carrier, airport)  # assuming predict_delay is your function
    
    # Append the prediction to the list
    predictions.append({
        'month': month,
        'carrier_name': carrier,
        'airport_name': airport,
        'predicted_arrival_delay': predicted_delay
    })

In [23]:
# Convert to DataFrame
predictions_df = pd.DataFrame(predictions)

In [24]:
# Save to CSV
predictions_df.to_csv('carrier_predicted_delays.csv', index=False)